# Direct Marketing with Amazon SageMaker Autopilot
---

*이 노트북은 [Direct Marketing with Amazon SageMaker Autopilot (영문 원본)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/autopilot/sagemaker_autopilot_direct_marketing.ipynb) 의 한국어 번역입니다.*


## Contents

1. [Introduction](#Introduction)
1. [Prerequisites](#Prerequisites)
1. [Downloading the dataset](#Downloading)
1. [Upload the dataset to Amazon S3](#Uploading)
1. [Setting up the SageMaker Autopilot Job](#Settingup)
1. [Launching the SageMaker Autopilot Job](#Launching)
1. [Tracking Sagemaker Autopilot Job Progress](#Tracking)
1. [Results](#Results)
1. [Cleanup](#Cleanup)

## Introduction

Amazon SageMaker Autopilot은 테이블 형식의 정형(tabular) 데이터셋을 위한 자동화된 머신 러닝 (일반적으로 AutoML 이라고 합니다) 솔루션입니다. 
현재 SageMaker Autopilot은 아래 기능들을 지원합니다.
* 테이블 형식의 입력 데이터; 자동 데이터 정제 및 전처리 지원
* 선형 회귀, 이진 분류, 및 다중 클래스 분류를 위한 자동 알고리즘 선택
* 자동 하이퍼 파라미터 최적화
* 분산 학습,
* 인스턴스 및 클러스터 크기 선택 자동 수행

이 노트북은 첫 번째 예시로서 AWS SDK를 사용하여 쉽게 머신 러닝 모델을 생성하고 배포하는 법을 보여줍니다.

"Hello World"에 해당하는 머신 러닝의 일반적인 입문 작업은 데이터셋을 사용하여 고객이 전화 통화 후 은행에 입금 계좌를 등록할지 여부를 예측하는 작업입니다. 관련 데이터셋에 대한 자세한 내용은 [Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing)을 참조하세요.

우편, 이메일, 전화 등을 통한 직접 마케팅은 고객 확보를 위한 일반적인 전술입니다. 리소스 및 고객의 관심이 제한되어 있으므로 목표는 특정 제안에 참여할 가능성이 있는 잠재 고객의 하위 집합만을 대상으로 하는 것입니다. 인구 통계, 과거 상호 작용 및 환경 요인과 같이 쉽게 이용할 수 있는 정보를 기반으로 잠재 고객을 예측하는 것은 일반적인 머신 러닝 문제입니다. 이 작업이 조직의 마케팅 리드 우선 순위 지정으로 쉽게 전환 될 것이라고 상상할 수 있습니다.


이 노트북은 이 데이터셋에서 Autopilot을 사용하여 여러 가지 잠재적인 옵션 또는 "후보자(candidates)"를 탐색하여 가장 정확한 ML 파이프라인을 얻는 방법을 보여줍니다. Autopilot에 의해 생성된 각 후보는 두 단계로 구성됩니다. 첫 번째 단계는 데이터셋에서 자동화된 피쳐 엔지니어링(feature engineering)을 수행하고 두 번째 단계는 알고리즘을 학습 및 조정하여 모델을 생성합니다. 이 모델을 배포할 때 비슷한 단계를 따릅니다. 리드(lead)가 추구 할 가치가 있는지 여부를 결정하기 위해 피쳐 엔지니어링과 추론(inference)이 뒤따릅니다. 이 노트북에는 모델 학습 방법과 리드 셋에서 배치 예측(batch predictions)을 수행하기 위해 모델을 배포하는 방법에 대한 지침이 포함되어 있습니다. 가능한 경우, high level SDK 인 Amazon SageMaker Python SDK를 사용하여 Amazon SageMaker를 쉽게 사용해 보세요.

다른 예제는 다양한 방법으로 모델을 사용자 정의하는 방법을 보여줍니다. 예를 들어, 디바이스에 배포된 모델에는 일반적으로 정확도(accuracy)뿐만 아니라 만족해야 하는 메모리 제약 조건이 있습니다. 다른 사용 사례에는 실시간 배포 요구 사항과 대기 시간(latency) 제약 조건이 있습니다. 


## Prerequisites

본 노트북을 시작하기 전에 다음을 수행하십시오.

- 학습 및 모델 데이터에 사용할 Amazon Simple Storage Service (Amazon S3) 버킷(bucket) 및 접두사(prefix)를 설정합니다. 이것은 Amazon SageMaker 학습과 동일한 리전(region) 내에 있어야 합니다. 아래 코드는 기본 버킷을 생성하거나 존재하는 경우 사용합니다.
- IAM 역할(role)에서 Autopilot이 데이터에 액세스할 수 있는 권한을 추가합니다. IAM 역할에 대한 자세한 내용은 Amazon SageMaker 설명서를 참조하십시오. https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html

In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-dm'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

## Downloading the dataset<a name="Downloading"></a>

University of California, Irvine ML 저장소에서 직접 마케팅 데이터셋을 다운로드하십시오:
[Download direct marketing dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing) 

In [2]:
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

local_data_path = './bank-additional/bank-additional-full.csv'


--2019-12-09 23:29:50--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444572 (434K) [application/x-httpd-php]
Saving to: ‘bank-additional.zip’

bank-additional.zip 100%[===================>] 434.15K  1.30MB/s    in 0.3s    

2019-12-09 23:29:51 (1.30 MB/s) - ‘bank-additional.zip’ saved [444572/444572]

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/bank-additional/
  inflating: __MACOSX/bank-additional/._.DS_Store  
  inflating: bank-additional/.Rhistory  
  inflating: bank-additional/bank-additional-full.csv  
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: __MACOSX/

## Upload the dataset to Amazon S3<a name="Uploading"></a>

데이터셋에서 Autopilot을 실행하기 전에 먼저 데이터셋을 검사하여 명백한 오류가 없는지 확인하세요. Autopilot 프로세스는 시간이 오래 걸릴 수 있으며 일반적으로 작업을 시작하기 전에 데이터셋을 검사하는 것이 좋습니다. 본 노트북의 데이터셋은 크기가 작기 때문에 노트북 인스턴스 자체에서 검사 할 수 있습니다. 만약 노트북 인스턴스 메모리에 맞지 않는 더 큰 데이터셋이 주어졌다면, Apache Spark와 같은 빅데이터 분석 도구를 사용하여 오프라인으로 데이터셋을 검사하십시오. [Deequ] (https://github.com/awslabs/deequ)는 Apache Spark를 기반으로 구축된 라이브러리로 대규모 데이터셋을 확인하는 데 도움이 됩니다. Autopilot은 최대 5GB의 데이터셋을 처리할 수 있습니다.

이제 Pandas 데이터프레임으로 데이터를 읽고 데이터를 간단히 살펴봅시다.

In [3]:
import pandas as pd

data = pd.read_csv(local_data_path, sep=';')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


종속변수(target variable, target column) 'y'를 예측하는 데 도움이 되는 20개 feature들이 있습니다.

Amazon SageMaker Autopilot은 데이터를 전처리합니다. 여러분은 결측값 처리, 범주형 피처를 수치형 피처로 변환, 데이터 스케일링 및 보다 복잡한 데이터 유형 처리와 같은 일반적인 데이터 전처리 기술을 수행할 필요가 없습니다.

또한 데이터셋을 학습셋 및 검증셋으로 분할할 필요가 없습니다. Autopilot이 이를 처리합니다. 

### Reserve some data for calling batch inference on the model

데이터를 학습셋 및 테스트셋으로 분리합니다. 학습셋은 SageMaker Autopilot에서 사용합니다. 테스트셋은 제안된 모델을 사용하여 추론(inference)을 수행하도록 예약되어 있습니다.

In [4]:
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['y'])

### Upload the dataset to Amazon S3

Amazon SageMaker 학습에서 사용할 수 있도록 파일을 .csv 형식으로 Amazon Simple Storage Service (Amazon S3)에 복사합니다.

In [5]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-1-143656149352/sagemaker/autopilot-dm/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-east-1-143656149352/sagemaker/autopilot-dm/test/test_data.csv


## Setting up the SageMaker Autopilot Job<a name="Settingup"></a>

Amazon S3에 데이터셋을 업로드한 후 Autopilot을 호출하여 이 데이터셋에서 모델을 학습하기 위한 최상의 ML 파이프라인을 찾을 수 있습니다.

Autopilot job 호출에 필요한 입력은 다음과 같습니다.

* 입력 데이터셋 및 모든 출력 아티팩트(artifacts)에 대한 Amazon S3 위치
* 예측하려는 데이터셋의 열 이름 (본 노트북의 경우 `y`)
* IAM 역할

현재 Autopilot은 CSV 형식의 테이블 형식(tabular) 데이터셋만 지원합니다. 모든 파일에 헤더 행(header row)이 있거나, 알파벳/어휘 순으로 정렬될 경우 데이터셋의 첫 번째 파일에 헤더 행이 있어야 합니다.

In [6]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'y'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

데이터셋으로 해결하려는 문제 유형 (회귀(`Regression`), 다중 클래스 분류(`MulticlassClassification`), 이진 분류(`BinaryClassification`))을 지정할 수도 있습니다. 확실하지 않은 경우 SageMaker Autopilot은 대상 열 (예측하려는 열)의 통계를 기반으로 문제 유형을 유추합니다.

여러분은 최대 파이프라인 평가 개수 또는 후보 개수를 제공하거나 (하나의 파이프라인 평가는 후보 모델을 생성하므로 `Candidate`라고 합니다.) 전체 Autopilot에 할당 된 총 시간을 제공하여 SageMaker Autopilot 작업의 실행 시간을 제한할 수 있습니다. 기본 설정에서 이 작업을 실행하는 데 약 4시간이 걸립니다. Autopilot은 최적의 학습 파라메터를 찾기 위해 사용하는 탐색 프로세스의 특성으로 인해 실행 시마다 실행 시간이 다릅니다.

## Launching the SageMaker Autopilot Job<a name="Launching"></a>

여러분은 이제 `create_auto_ml_job` API를 호출하여 Autopilot 작업을 시작할 수 있습니다.

In [7]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-banking-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      RoleArn=role)

AutoMLJobName: automl-banking-09-23-29-57


{u'AutoMLJobArn': u'arn:aws:sagemaker:us-east-1:143656149352:automl-job/automl-banking-09-23-29-57',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '97',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 09 Dec 2019 23:29:56 GMT',
   'x-amzn-requestid': '0f4a87e6-260f-484b-9023-b1db9a1b31ee'},
  'HTTPStatusCode': 200,
  'RequestId': '0f4a87e6-260f-484b-9023-b1db9a1b31ee',
  'RetryAttempts': 0}}

## Tracking SageMaker Autopilot job progress<a name="Tracking"></a>

SageMaker Autopilot job은 다음과 같은 high-level 단계들로 구성됩니다.

* 데이터 분석(Analyzing Data): 데이터셋이 분석되고 Autopilot은 데이터셋에서 시도해야 하는 ML 파이프라인 목록을 제공합니다.
* 피쳐 엔지니어링(Feature Engineering): Autopilot이 데이터셋의 개별 피처 및 집계(aggregate) 레벨에서 피처 변환을 수행합니다.
* 모델 튜닝(Model Tuning): 가장 우수한 성능의 파이프라인은 학습 알고리즘(파이프라인의 마지막 단계)에 대한 최적의 하이퍼파라미터와 함께 선택됩니다.

In [ ]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProg

## Results

이제 `describe_auto_ml_job` API를 사용하여 SageMaker Autopilot job에서 선택한 최상의 후보를 찾아 보세요.

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{u'CandidateStatus': u'Completed', u'ObjectiveStatus': u'Succeeded', u'CandidateSteps': [{u'CandidateStepType': u'AWS::SageMaker::ProcessingJob', u'CandidateStepName': u'automl-banking-09-23-29-57-db-1-7eaa84b299cf4d05ad0b911c158be27', u'CandidateStepArn': u'arn:aws:sagemaker:us-east-1:143656149352:processing-job/automl-banking-09-23-29-57-db-1-7eaa84b299cf4d05ad0b911c158be27'}, {u'CandidateStepType': u'AWS::SageMaker::TrainingJob', u'CandidateStepName': u'automl-banking-09-23-29-57-automl-ban-dpp7-1-0f3a9010dd16422c9f', u'CandidateStepArn': u'arn:aws:sagemaker:us-east-1:143656149352:training-job/automl-banking-09-23-29-57-automl-ban-dpp7-1-0f3a9010dd16422c9f'}, {u'CandidateStepType': u'AWS::SageMaker::TransformJob', u'CandidateStepName': u'automl-banking-09-23-29-57-automl-ban-dpp7-rpb-1-8b97923466304e', u'CandidateStepArn': u'arn:aws:sagemaker:us-east-1:143656149352:transform-job/automl-banking-09-23-29-57-automl-ban-dpp7-rpb-1-8b97923466304e'}, {u'CandidateStepType': u'AWS::SageMake

### Perform batch inference using the best candidate

데이터셋에서 SageMaker Autopilot job을 완료했으므로 추론 파이프라인([Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html))을 사용하여 후보에서 모델을 생성하세요.

In [ ]:
model_name = 'automl-banking-model-' + timestamp_suffix

model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

Model ARN corresponding to the best candidate is : arn:aws:sagemaker:us-east-1:143656149352:model/automl-banking-model-09-23-29-57


Amazon SageMaker 배치 변환(batch transform)을 사용하여 배치 유추(batch inference)를 사용할 수 있습니다. Amazon SageMaker 호스팅을 사용하여 온라인 추론을 수행하기 위해 동일한 모델을 배포할 수도 있습니다.

In [ ]:
transform_job_name = 'automl-banking-transform-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': test_data_s3_path
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources
)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '113',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 10 Dec 2019 01:36:26 GMT',
   'x-amzn-requestid': 'f2f10cb5-29d6-4833-9fd4-60a0e05ce4f2'},
  'HTTPStatusCode': 200,
  'RequestId': 'f2f10cb5-29d6-4833-9fd4-60a0e05ce4f2',
  'RetryAttempts': 0},
 u'TransformJobArn': u'arn:aws:sagemaker:us-east-1:143656149352:transform-job/automl-banking-transform-09-23-29-57'}

변환 작업이 완료되었는지 확인하세요.

In [ ]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


이제 변환 작업의 결과를 확인해 보세요.

In [ ]:
s3_output_key = '{}/inference-results/test_data.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=';')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,no
0,no
1,no
2,no
3,no
4,no
...,...
8232,yes
8233,yes
8234,no
8235,yes


### View other candidates explored by SageMaker Autopilot

여러분은 SageMaker Autopilot에서 탐색한 모든 후보들을(서로 다른 하이퍼파라미터 조합을 가진 파이프라인 평가) 확인하고 최종 성능 메트릭별로 정렬할 수 있습니다.

In [ ]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
  print (str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))
  index += 1

1  automl-banki-tuning-job-1-24abf-179-94592c24  0.916224002838
2  automl-banki-tuning-job-1-24abf-219-85d924e2  0.915009975433
3  automl-banki-tuning-job-1-24abf-197-4c74475a  0.91455501318
4  automl-banki-tuning-job-1-24abf-077-fbe5687e  0.914403021336
5  automl-banki-tuning-job-1-24abf-032-e73d55cc  0.914403021336
6  automl-banki-tuning-job-1-24abf-074-19b223cb  0.914098978043
7  automl-banki-tuning-job-1-24abf-243-10d10656  0.913946986198
8  automl-banki-tuning-job-1-24abf-057-9942308d  0.913946986198
9  automl-banki-tuning-job-1-24abf-216-2fcabe5b  0.913644015789
10  automl-banki-tuning-job-1-24abf-227-11f82b7e  0.913644015789


### Candidate Generation Notebook
    
Sagemaker AutoPilot은 후보 정의(Candidate Definitions) 노트북을 자동 생성합니다. 이 노트북은 Sagemaker Autopilot이 수행한 다양한 단계를 대화식으로 단계별로 실행하여 최상의 후보에 도달할 수 있습니다. 이 노트북은 또한 병렬 처리, 사용된 하드웨어, 탐색된 알고리즘, feature 추출 스크립트 등과 같은 다양한 런타임 매개 변수를 대체하는 데 사용될 수 있습니다.

이 노트북은 아래 Amazon S3 경로에서 다운로드할 수 있습니다.

In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']

u's3://sagemaker-us-east-1-143656149352/sagemaker/autopilot-dm/output/automl-banking-09-23-29-57/sagemaker-automl-candidates/automl-banking-09-23-29-57-pr-1-b46aa4cc9a7845ea97b78da4be81248/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb'

### Data Exploration Notebook

Sagemaker Autopilot은 아래 Amazon S3 경로에서 다운로드할 수있는 데이터 탐색(Data Exploration) 노트북을 자동 생성합니다.

In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']

u's3://sagemaker-us-east-1-143656149352/sagemaker/autopilot-dm/output/automl-banking-09-23-29-57/sagemaker-automl-candidates/automl-banking-09-23-29-57-pr-1-b46aa4cc9a7845ea97b78da4be81248/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb'

## Cleanup

Autopilot job은 데이터셋 분할, 전처리 스크립트 또는 전처리된 데이터 등과 같은 많은 아티팩트들을 작성합니다. 아래 셀을 주석 처리하지 않으면 해당 아티팩트들을 삭제합니다. 이 작업은 생성된 모든 모델들과 자동 생성된 노트북들도 삭제합니다.

In [ ]:
#s3 = boto3.resource('s3')
#bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix,auto_ml_job_name)
#bucket.objects.filter(Prefix=job_outputs_prefix).delete()